In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import math

feature = {i:label for i,label in zip(
                range(4),
                  ('sepal length in cm',
                  'sepal width in cm',
                  'petal length in cm',
                  'petal width in cm', ))
               }

In [2]:
#데이터 만들기

df = pd.io.parsers.read_csv(
    filepath_or_buffer='https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
    header=None,
    sep=',',
    )
df.columns = [j for i,j in sorted(feature.items())] + ['class label']
df.dropna(how="all", inplace=True)

df.tail()

,sepal length in cm,sepal width in cm,petal length in cm,petal width in cm,class label
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
149,5.9,3.0,5.1,1.8,Iris-virginica


In [96]:
def LDA(df):
    X = df.iloc[:,0:4].values
    y = df['class label'].values

    enc = LabelEncoder()
    label_encoder = enc.fit(y)
    y = label_encoder.transform(y) + 1

    label_dict = {1: 'Setosa', 2: 'Versicolor', 3:'Virginica'}
    
    #d차원 mean벡터 구하기
    np.set_printoptions(precision=4)

    mean_vectors = []
    for cl in range(1,4):
        mean_vectors.append(np.mean(X[y==cl], axis=0))

    #분산 matrix 계산
    S_W = np.zeros((4,4)) #클래스 간 분산
    for cl,mv in zip(range(1,4), mean_vectors):
        class_sc_mat = np.zeros((4,4))                  
        for row in X[y == cl]:
            row, mv = row.reshape(4,1), mv.reshape(4,1) #shape 맞추고
            class_sc_mat += (row-mv).dot((row-mv).T) #내적
        S_W += class_sc_mat                             
    print('within-class Scatter Matrix:\n', S_W)

    overall_mean=np.mean(X,axis=0)

    S_B=np.zeros((4,4)) #클래스 내 분산
    for i,mean_vec in enumerate(mean_vectors):
        n=X[y==i+1,:].shape[0]
        mean_vec=mean_vec.reshape(4,1) #shape 맞추기
        overall_mean=overall_mean.reshape(4,1) #shape 맞추기
        S_B+=n*(mean_vec - overall_mean).dot((mean_vec - overall_mean).T) #내적

    print('between-class Scatter Matrix:\n',S_B)

    #아이젠 value 계산
    
    eig_vals, eig_vecs=np.linalg.eig(np.linalg.inv(S_W).dot(S_B))

    for i in range(len(eig_vals)):
        eigvec_sc=eig_vecs[:,i].reshape(4,1)
        print('\nEigenVector {} : \n{}'.format(i+1, eigvec_sc.real))
        print('Eigenvalue{:}:{:.2e}'.format(i+1,eig_vals[i].real))

    #새로운 feature subspace에 대한 linear discriminants 선택 
    eig_pairs=[(np.abs(eig_vals[i]),eig_vecs[:,i]) for i in range(len(eig_vals))]
    eig_pairs=sorted(eig_pairs, key=lambda k: k[0], reverse=True)

    eigv_sum = sum(eig_vals)
    for i,j in enumerate(eig_pairs):
        print('eigenvalue {0:}: {1:.2%}'.format(i+1, (j[0]/eigv_sum).real))
        
LDA(df)

within-class Scatter Matrix:
 [[38.9562 13.683  24.614   5.6556]
 [13.683  17.035   8.12    4.9132]
 [24.614   8.12   27.22    6.2536]
 [ 5.6556  4.9132  6.2536  6.1756]]
between-class Scatter Matrix:
 [[ 63.2121 -19.534  165.1647  71.3631]
 [-19.534   10.9776 -56.0552 -22.4924]
 [165.1647 -56.0552 436.6437 186.9081]
 [ 71.3631 -22.4924 186.9081  80.6041]]

EigenVector 1 : 
[[-0.2049]
 [-0.3871]
 [ 0.5465]
 [ 0.7138]]
Eigenvalue1:3.23e+01

EigenVector 2 : 
[[-0.009 ]
 [-0.589 ]
 [ 0.2543]
 [-0.767 ]]
Eigenvalue2:2.78e-01

EigenVector 3 : 
[[-0.8844]
 [ 0.2854]
 [ 0.258 ]
 [ 0.2643]]
Eigenvalue3:3.42e-15

EigenVector 4 : 
[[-0.2234]
 [-0.2523]
 [-0.326 ]
 [ 0.8833]]
Eigenvalue4:1.15e-14
eigenvalue 1: 99.15%
eigenvalue 2: 0.85%
eigenvalue 3: 0.00%
eigenvalue 4: 0.00%
